
# 作業一：Prompt Engineering 實作

> 請「先」在檔名中填入你的學號再上傳（在下方 **[提交與輸出]** 有自動命名）。  
> 完成後請依課程規定提交。

---

## 規劃與來源
本 Notebook 依據課程投影片之三個任務設計：
- 任務一：Email 格式轉換（角色設定、格式控制、語氣調整）  
- 任務二：文章摘要（使用 Chain-of-Thought）  
- 任務三：翻譯與語氣調整（多重指令整合、文化語境）  

請根據各章節中的 **TODO** 完成核心實作。助教已替你準備好輸入範例、評分提示、與基本工具函式。


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



## 注意事項與遲交規則（節錄）
- 遲交扣分：每延遲一天扣 **10%**，最多寬限 **3** 天；超過不受理，記 **0** 分。  
- 以繳交系統時間為準。  
- 請保留你的設計思路（尤其是任務二的 CoT 步驟）。

> 提醒：好的 Prompt = 清楚、具體、有條理。



## 0. 環境與套件
> 本模板預留了與 LLM 互動的介面，但**不綁定特定供應商**。你可以選擇 OpenAI、Anthropic、Azure OpenAI、Google 等。  
> 若你使用 Colab，請先安裝對應的 Python SDK（例如 `openai`）。


In [ ]:
# === 可選：安裝你要用的 SDK（Colab 環境）===
# 例如：
!pip -q install openai

import os
from dataclasses import dataclass
from typing import Optional, Dict, Any

# ========== 可重用：統一的 LLM 呼叫介面 ==========
@dataclass
class LLMConfig:
    provider: str = "openai"
    model: str = "gpt-5-nano"     # TODO: 指定模型名稱（必填）


def call_llm(prompt: str, system: Optional[str]=None, cfg: Optional[LLMConfig]=None, **kwargs) -> str:
    """
    統一的 LLM 呼叫函式（簡化版）。
    - 你可以在此實作實際的 SDK 呼叫；或在各任務中自行呼叫不同供應商的 API。
    - 若不想用雲端 API，可改為你自定義的規則程式碼（但需在報告中說明限制）。
    """
    cfg = cfg or LLMConfig()
    provider = cfg.provider.lower()
    os.environ["OPENAI_API_KEY"] = ""  # TODO: 對應的 API Key 環境變數名

    # TODO: 依你選定的 provider 完成實作（下方示例為 OpenAI 的「佔位」程式碼片段）
    # ===== OpenAI （需: pip install openai）=====
    if provider == "openai":
        from openai import OpenAI
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        messages = []
        if system:
            messages.append({ "role": "system", "content": system })
        messages.append({ "role": "user", "content": prompt })
        resp = client.chat.completions.create(model=cfg.model, messages=messages, **kwargs)
        return resp.choices[0].message.content
        raise NotImplementedError("TODO: 請在 call_llm() 中實作 OpenAI呼叫。")
    else:
        raise NotImplementedError(f"TODO: 尚未支援 provider={provider!r}。")


In [ ]:

# ========== 共用工具 ==========
def count_zh_chars(text: str) -> int:
    """簡易字數統計（中文為主，含中英文混排時可視需要自行調整）。"""
    return len(text.strip())

def save_text(path: str, content: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)

def load_text(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()



---
# 任務一：Email 格式轉換

**學習重點**：角色設定、格式控制、語氣調整  
**輸入範例**：
> 嘿老闆，明天10點的會議我可能會遲到一下，大概15分鐘吧，昨晚小孩發燒，早上得先送他去看醫生。會議的資料我都準備好了，可以先請小王開始報告嗎？

**任務要求**：
- 將上述訊息改寫成**正式商業郵件**
- 必須包含：**主旨、稱謂、清楚說明、解決方案、專業結尾**
- **保持原意**但提升專業度
- 在 Prompt 中**展示你的設計思路**（例如列出需求、格式、語氣等）

**評分重點**：Prompt 清晰度與結構性、格式控制是否成功、語氣轉換自然度

**學習過程紀錄**：在這次 Email 格式轉換的任務中，我嘗試了多種不同的 Prompt。起初讓 AI 直接改寫，但輸出內容常夾雜英文或出現教學式標題，顯得不自然。後來我在 Prompt 中明確規定信件格式、語氣與署名，並加上「禁止英文詞彙與括號編號」等條件。最終版本效果最好，信件語氣正式流暢、格式清楚，內容可直接寄出。過程中最大的困難是 AI 容易混入英文，我透過限制指令與明確範例成功解決，使輸出結果符合正式商業郵件的要求。

In [ ]:

# ===== TODO(學生)：設計你的 Prompt（可含 system + user）=====
email_raw = (
    "嘿老闆，明天10點的會議我可能會遲到一下，大概15分鐘吧，"
    "昨晚小孩發燒，早上得先送他去看醫生。會議的資料我都準備好了，"
    "可以先請小王開始報告嗎？"
)

system_email = """
你是一位專業的商務書信撰寫專家。
請將口語化訊息改寫成正式、清楚、可直接寄出的中文商業郵件。

風格要求：
- 用詞禮貌、語氣自然、結構完整。
- 以正式郵件格式輸出，包含以下四個部分（依序）：
  1）主旨：簡潔明確。
  2）稱謂：例如「老闆您好，」
  3）內文：1～4段，說明原因與安排。
  4）結尾：禮貌收尾與簽名（署名為「劉于岑」）。
- 不可出現英文字母、括號、翻譯說明或數字編號。
- 文字風格應自然、專業，符合職場郵件書寫習慣。
- 禁止輸出任何英文字母或外語詞彙（例如：him、English 等）。
- 所有內容須以自然中文撰寫，格式符合正式郵件慣例。
- 信件段落前面不要有標題（例如：稱謂：、內文：、結尾：等）
""".strip()

# === 在此撰寫你的 user prompt，需明確描述格式要求與轉換目標 ===
user_email_prompt = f"""
請將以下內容改寫為正式中文商業郵件：
請保持原意：我明天10點的會議可能會遲到15分鐘，
因為孩子生病要先帶他去看醫生，資料已經準備好，
希望可請小王先開始報告。

【原始訊息】
{email_raw}
"""

# === 呼叫 LLM（或你自定義的規則程式）===
# TODO: 取消下一行註解，並完成 call_llm 的實作
email_formal = call_llm(user_email_prompt, system=system_email)

# 儲存輸出
save_text("outputs/task1_email_formal.txt", email_formal)
print("✅ 任務一完成（暫存）。輸出檔：outputs/task1_email_formal.txt")

✅ 任務一完成（暫存）。輸出檔：outputs/task1_email_formal.txt



---
# 任務二：文章摘要（使用 CoT）

**輸入文章（約 200 字）**：
> 根據最新研究顯示，台灣的科技產業正面臨人才短缺的嚴峻挑戰。調查發現，有超過70%的科技公司表示難以找到合適的AI和資料科學人才。這個問題的根源包括：教育體系與產業需求脫節、薪資競爭力不足，以及人才外流至海外市場。為了解決這個問題，政府推出了多項措施，包括增加大學相關科系名額、提供企業培訓補助，以及放寬外籍人才引進政策。然而，專家認為這些措施需要時間才能看到成效，短期內企業仍需要自行投資人才培育。

**任務要求**：
- 使用 **Chain-of-Thought** 方法設計 Prompt
- **明確展示分析步驟**（例：主題識別 → 要點提取 → 重要性排序 → 總結）
- 產生 **50 字以內** 的精簡摘要（需保留關鍵資訊）

**評分重點**：
- CoT 步驟的邏輯性與完整性
- 摘要的精確度與資訊保留度
- 是否展現結構化思考過程

**學習過程紀錄**：在文章摘要任務中，我一開始讓 AI 直接生成摘要，但內容太簡略，看不出思考過程。之後我加入 Chain-of-Thought 的提示，請 AI 依序列出主題識別、要點提取與重要性排序。不過最初我讓 AI 用「最重要：次要：次次要：」的方式標示，結果閱讀起來不清楚，因此我改成用數字條列，讓層次更明確。最後我再新增「總結」步驟，讓 AI 先歸納重點再寫出 50 字內的摘要。這個版本的輸出邏輯清楚、層次分明，摘要內容也完整且符合任務要求。

In [ ]:

article = (
    """根據最新研究顯示，台灣的科技產業正面臨人才短缺的嚴峻挑戰。調查發現，有超過70%的科技公司表示難以找到合適的AI和資料科學人才。這個問題的根源包括：教育體系與產業需求脫節、薪資競爭力不足，以及人才外流至海外市場。為了解決這個問題，政府推出了多項措施，包括增加大學相關科系名額、提供企業培訓補助，以及放寬外籍人才引進政策。然而，專家認為這些措施需要時間才能看到成效，短期內企業仍需要自行投資人才培育。"""
)

# ===== TODO(學生)：設計你的 CoT Prompt =====
system_cot = """
你是一位中文摘要撰寫助教。請使用「分析步驟」思考，最後再產生 50 字內的摘要。
請用以下格式回答：

[分析步驟]
1）主題識別：說明文章的主要議題或核心問題。
2）要點提取：列出關鍵資訊（3~4項），例如原因、對策或影響。
3）重要性排序：說明哪些是最重要的重點，用列點的方式（例如：1. 、2. 、3. 等）。
4）總結：根據以上內容，統整出可作為摘要依據的核心句。
[最終摘要]（≤ 50 字）
根據分析步驟，請輸出最簡潔的中文摘要。
""".strip()

user_cot_prompt = f"""
請閱讀以下文章，依指示逐步分析，最後產生不超過50字的摘要：
【文章】
{article}
"""

# TODO: 使用 LLM（或你自定義的規則程式）產生輸出
cot_output = call_llm(user_cot_prompt, system=system_cot)

# 50字內檢查（僅對「最終摘要」行做粗略偵測）
lines = [ln for ln in cot_output.splitlines() if ln.strip()]
final_lines = [ln for ln in lines if ln.strip().startswith("[最終摘要]") or ln.strip().startswith("最終摘要")]
if final_lines:
    final = final_lines[-1].replace("[最終摘要]", "").strip("：: ")
    n_chars = count_zh_chars(final)
    print(f"最終摘要字數：約 {n_chars} 字（目標 <= 50）")
else:
    print("⚠️ 未偵測到最終摘要段落，請檢查輸出格式。")

save_text("outputs/task2_summary_cot.txt", cot_output)
print("✅ 任務二完成（暫存）。輸出檔：outputs/task2_summary_cot.txt")

最終摘要字數：約 8 字（目標 <= 50）
✅ 任務二完成（暫存）。輸出檔：outputs/task2_summary_cot.txt



---
# 任務三：翻譯與語氣調整（多重指令）

**輸入句子**：  
> I'm afraid I cannot attend tomorrow's meeting due to a scheduling conflict. Would it be possible to reschedule to next week?

**任務要求**：
- 翻譯成**繁體中文**
- 調整為**友善、輕鬆**的語氣（像和好朋友說話）
- 保持原意並加入台灣常用語氣
- **設計一個 Prompt** 能同時完成以上所有要求  
**進階加分**：
- 讓 AI 提供 **2–3 種不同語氣程度** 的版本，並**說明各版本適用情境**。

**學習過程紀錄**：在翻譯與語氣調整的任務中，我在設計 Prompt 時，嘗試讓 AI 同時做到翻譯、語氣調整與多版本輸出。一開始我只讓它翻譯成繁體中文，但語氣太正式、不自然，後來我加入「友善、輕鬆、像和朋友講話」等關鍵字，AI 的輸出就變得更貼近台灣口語。之後我再進一步要求它產生三種語氣（正式、日常、超口語），並加上適用情境說明，讓結果更有層次、也方便比較。過程中最大的困難是找到能同時涵蓋多項指令的 Prompt，但透過多次修改語氣提示，最終能讓 AI 生成自然又符合任務需求的翻譯結果。

In [ ]:

en_sentence = "I'm afraid I cannot attend tomorrow's meeting due to a scheduling conflict. Would it be possible to reschedule to next week?"

# ===== TODO(學生)：設計能同時完成「翻譯＋語氣調整」的 Prompt =====
system_trans = """
你是一位了解台灣用語與語氣的翻譯專家。
請將英文句子翻譯成繁體中文，語氣友善、自然、輕鬆，
同時保留原意與清楚度。
請提供 2～3 種不同語氣程度（例如：正式、日常、超口語）版本，
並在每個版本後說明適合的使用情境。
""".strip()

user_trans_prompt = f"""
請將以下英文句子翻譯成繁體中文，語氣友善輕鬆，保留原意：
{en_sentence}

請提供三個版本，語氣分別為：
1. 較正式但仍親切
2. 自然日常（朋友之間的語氣）
3. 超口語、輕鬆聊天式
並在每個版本後加一句說明，說明這種語氣適合用在哪種情境。
""".strip()

# TODO: 使用 LLM 產生輸出
trans_output = call_llm(user_trans_prompt, system=system_trans)

save_text("outputs/task3_translation_tone.txt", trans_output)
print("✅ 任務三完成（暫存）。輸出檔：outputs/task3_translation_tone.txt")

✅ 任務三完成（暫存）。輸出檔：outputs/task3_translation_tone.txt



---
## 提交與輸出
請在下方設定你的 **學號** 與 **姓名**，系統會將三個任務的輸出合併成一個檔案，並以 `學號_prompt_hw1.txt` 命名，方便上傳。

> 若你實作了實際的 API 呼叫，請同時上傳 `.ipynb` 與最終 `.txt`，並確保不要把 API Key 放進檔案。


In [ ]:

# ===== TODO(學生)：填入你的資訊 =====
student_id = "411278055"   # 例：學號
student_name = "劉于岑"

bundle_name = f"{student_id}_prompt_hw1.txt"
out_paths = [
    "outputs/task1_email_formal.txt",
    "outputs/task2_summary_cot.txt",
    "outputs/task3_translation_tone.txt",
]

merged = []
for p in out_paths:
    if os.path.exists(p):
        merged.append(f"===== {os.path.basename(p)} =====\n" + load_text(p) + "\n")
    else:
        merged.append(f"===== {os.path.basename(p)} =====\n(尚未產生)\n")

save_text(os.path.join("outputs", bundle_name), "\n".join(merged))
print(f"✅ 已整合輸出：outputs/{bundle_name}")


✅ 已整合輸出：outputs/411278055_prompt_hw1.txt



---
## 誠信與說明
- 請在 Notebook 中保留你的 Prompt 設計思路與步驟（尤其是任務二的 CoT）。
- 若使用了特定外部資源或工具，請於最終輸出結尾簡述。
- 請勿外流你的 API Key。建議以環境變數或本機設定檔管理。

祝順利完成作業！
